# 6 Pandasを使ったデータ加工処理

- **[6.1 この章の概要について](#6.1-この章の概要について)**
<br><br>
- **[6.2 Pandasの基本的なデータ操作](#6.2-Pandasの基本的なデータ操作)**
    - [6.2.1 階層型インデックス](#6.2.1-階層型インデックス)
    - [6.2.2 データの結合](#6.2.2-データの結合)
    - [6.2.3 データの操作と変換](#6.2.3-データの操作と変換)
    - [6.2.4 データの集約とグループ演算](#6.2.4-データの集約とグループ演算)
<br><br>
- **[6.3 欠損データと異常値の取り扱いの基礎](#6.3-欠損データと異常値の取り扱いの基礎)**
    - [6.3.1 欠損データの扱い方](#6.3.1-欠損データの扱い方)
    - [6.3.2 異常データの扱い方](#6.3.2-異常データの扱い方)
<br><br>
- **[6.4 時系列データの取り扱いの基礎](#6.4-時系列データの取り扱いの基礎)**
    - [6.4.1 時系列データの処理と変換](#6.4.1-時系列データの処理と変換)
    - [6.4.2 移動平均](#6.4.2-移動平均)
<br><br>
- **[6.5 総合問題](#6.5-総合問題)**
    - [6.5.1 総合問題1](#6.5.1-総合問題1)

巻末参考URL : https://docs.google.com/spreadsheets/d/e/2PACX-1vRXG56iX93yr7gCO-9AStIhA8YXM4eoYoL_VPnT-SRTg7AFY_SL3uticBtQJvSMtIMXr_bwhqPeEZbh/pubhtml

***

## 6.1 データの加工操作
この章ではPandasを使ったデータ加工処理について、もう少し詳しく学んでいきます。Pandasは2章で学んだように、ある条件を満たすデータを抽出したり、操作するなど、さまざまな機能があります。

たとえば、全国の小学校で同じ算数のテストを実施したケースを考えます。
それぞれの都道府県の最高点取得者だけを抜き出したいケースもありますし、それぞれの都道府県の平均点を出したい場合等、さまざまな集計軸があります。さらに、都道府県×学校×クラスの3軸で平均値を算出したい場合や、さらに男女で計算したい場合など、軸が複数になっているケースもあります。Pandasを使えば、そのような集計をすることもできます。また、他のデータ（たとえば、国語の試験結果）とつなげたいときも、キー（各学生に与えられた一意となるデータなど）があれば、結合して1つの`DataFrame`にして、まとめて処理することもできます。

ほか、時系列データを扱うときもPandasは役に立ちます。
たとえば、ある店舗の日時の売上推移データを取り扱うときに、1週間や1か月ごとの平均値の推移を簡単に計算することができます。これらのコーディングを一から記述するとなると大変ですが、Pandasではこのような計算も1～2行ほどのコードを書くだけで実行することができます。さらに、データに欠損値や何か異常値が入っているとき、それらを何らかの方法で一括処理したい場合に使えます。

もちろん、これらの処理は自分で、いちからPythonのプログラムを書くことで対応できますが、実装するのに時間がかかります。Pandasの機能を使うと簡単に操作できます。あと、 機械学習のモデルを構築するときは、そのアルゴリズムが使えるようにデータを前処理する必要があります。たとえば、縦に並んでいたデータのカラムを横に並べたい場面などもあり、そういった操作もPandasで簡単にできます。

上記のようなデータ操作をする場合、SQLやエクセルのピボットテーブルなどを使っても処理できますが、Pythonのプログラムだけで一貫してコーディングしたい場合はPandasを使うと便利です。
なお、Pandasではグラフの描写機能もあり、ハンドリングしたデータをグラフとしてすぐに描写できます。このデータのグラフ化については次の7章でみていくことにします。

### 6.1.1　この章で使うライブラリ

この章では、2章で紹介した各種ライブラリを使います。次のようにインポートすることを前提として、以下、進めていきます。

In [2]:
# 以下のライブラリを使うので、あらかじめ読み込んでおいてください
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 小数第３位まで表示
%precision 3

'%.3f'

## 6.2 Pandasの基本的なデータ操作
ゴール：Pandasの階層型インデックスを使える、データの結合ができる、group byなどを使って集計処理ができる

まずは、Pandasの基本的なデータ操作から始めます。

### 6.2.1 階層型インデックス
キーワード：階層型インデックス


データを複数軸で集計したいというモチベーションがあるとき、設定すると便利なのが**階層型インデックス**です。
2章でPandasのインデックスについて少し扱いましたが、インデックスとは索引やラベルのようなイメージです。2章では、1つのインデックスだけで扱いましたが、この章の冒頭で説明したように、複数の軸で階層的にインデックスを設定したいこともあります。階層的にインデックスを設定することで、各階層ごとに集計が可能になり、後々便利です。

次に示すデータセットは、インデックスを2段構造で設定した例です。インデックスを設定するには、`index`の部分にその値を指定します。この例では、はじめが`a`と`b`、次が`1`と`2`でインデックスを設定しています。

In [21]:
hier_data_frame = DataFrame(np.arange(9).reshape((3,3))
                           ,index = [['a','a','b'],[1,2,2]]
                           ,columns = [['Osaka','Tokyo','Osaka']
                                      ,['Blue','Red','Red']]
                           )
hier_data_frame

Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

これらのインデックスやカラムには、名前をつけることもできます。

In [22]:
# indexに名前を付ける
hier_data_frame.index.names =['key1','key2']
# カラムに名前を付ける
hier_data_frame.columns.names =['city','color']
hier_data_frame

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5
b    2        6     7     8

#### カラムの絞り込み

ここでたとえば、カラムの`city`について、「`Osaka`」のデータだけ見たいとしましょう。次のようにすると、グループの絞り込みができます。

In [23]:
hier_data_frame['Osaka']

color      Blue  Red
key1 key2           
a    1        0    2
     2        3    5
b    2        6    8

#### インデックスを軸にした集計

次はあるインデックスを軸にした集計の例てす。以下の例は、`key2`ごとに合計を計算するものです。

In [24]:
# 階層ごとの要約統計量：行合計
hier_data_frame.sum(level='key2')

city,Osaka,Tokyo,Osaka
color,Blue,Red,Red
key2,,,
1,0,1,2
2,9,11,13


同様にして、「`color`」ごとの合計値は、次のようにして計算できます。軸は`axis`で設定します。

In [25]:
# 列合計
hier_data_frame.sum(level='color',axis=1)

color      Blue  Red
key1 key2           
a    1        0    3
     2        3    9
b    2        6   15

#### インデックスの要素の削除

あるインデックスを削除したい場合は、`drop`を使います。`drop`を使うと、インデックスの要素を削除できます。次の例では、`key1`の`b`を削除しています。

In [26]:
hier_data_frame.drop(["b"])

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5

#### <練習問題 1>

次のデータに対して、`Kyoto`の列だけ抜き出してみましょう。

In [27]:
hier_data_frame1 = DataFrame(np.arange(12).reshape((3,4))
                           ,index = [['c','d','d'],[1,2,1]]
                           ,columns = [['Kyoto','Nagoya','Hokkaido','Kyoto']
                                      ,['Yellow','Yellow','Red','Blue']]
                           )

hier_data_frame1.index.names =['key1','key2']
hier_data_frame1.columns.names =['city','color']
hier_data_frame1

city       Kyoto Nagoya Hokkaido Kyoto
color     Yellow Yellow      Red  Blue
key1 key2                             
c    1         0      1        2     3
d    2         4      5        6     7
     1         8      9       10    11

#### <練習問題 2>

練習問題1のデータに対して、`city`をまとめて列同士の平均値を出してください。

#### <練習問題 3>

練習問題1のデータに対して、`key2`ごとに行の合計値を算出してみましょう。


### 6.2.2 データの結合
キーワード：内部結合、外部結合、縦結合

データの結合については2章で少し学びました。データを結合したいケースは多々あり、データをつなげることで集計がしやすくなったり、新しい軸における値がわかったりします。ぜひ、マスターしてください。
ただし、結合と言っても、さまざまなパターンがあります。以下でそれらを紹介していきます。

まずは、この節でサンプルとして使うマージの対象となるデータを準備します。ここでは次に提示する`attri_data1`（以下、データ1）と`attri_data2`（以下、データ2）の2つのテータを使います。

In [28]:
# データ1の準備
attri_data1 = {'ID':['100','101','102','103','104','106','108','110','111','113']
        ,'city':['Tokyo','Osaka','Kyoto','Hokkaido','Tokyo','Tokyo','Osaka','Kyoto','Hokkaido','Tokyo']
        ,'birth_year':[1990,1989,1992,1997,1982,1991,1988,1990,1995,1981]
        ,'name':['Hiroshi','Akiko','Yuki','Satoru','Steeve','Mituru','Aoi','Tarou','Suguru','Mitsuo']}
attri_data_frame1 = DataFrame(attri_data1)
attri_data_frame_index1 = DataFrame(attri_data1,index=['e','b','a','d','c','f','g','h','j','i'])
attri_data_frame_index1

,ID,birth_year,city,name
e,100,1990,Tokyo,Hiroshi
b,101,1989,Osaka,Akiko
a,102,1992,Kyoto,Yuki
d,103,1997,Hokkaido,Satoru
c,104,1982,Tokyo,Steeve
f,106,1991,Tokyo,Mituru
g,108,1988,Osaka,Aoi
h,110,1990,Kyoto,Tarou
j,111,1995,Hokkaido,Suguru
i,113,1981,Tokyo,Mitsuo


In [29]:
# データ2の準備
attri_data2 = {'ID':['100','101','102','105','107']
        ,'math':[50,43,33,76,98]
        ,'English':[90,30,20,50,30]
        ,'sex':['M','F','F','M','M']
        ,'index_num':[0,1,2,3,4]}
attri_data_frame2 = DataFrame(attri_data2)
attri_data_frame2

,English,ID,index_num,math,sex
0,90,100,0,50,M
1,30,101,1,43,F
2,20,102,2,33,F
3,50,105,3,76,M
4,30,107,4,98,M


### 結合

では、この2つのデータを結合する方法を見ていきましょう。
データ1とデータ2を結合する方法は、次の4パターンが考えられます。

①内部結合（INNER JOIN）
両方にキーがある場合に結合します。

②全結合（FULL JOIN）
どちらかにキーがある場合に結合します。

③左外部結合（LEFT JOIN）
左側にあるデータのキーがある時に結合します。

④右外部結合（RIGHT JOIN）
右側にあるデータのキーがある時に結合します。

ここでは主に、「内部結合」と「（左）外部結合」を使います。この2つを理解しておいてください。

![comment](http://www.dofactory.com/Images/sql-joins.png)
参照URL:http://www.dofactory.com/Images/sql-joins.png

#### 内部結合
上記のデータ2つに対して、`ID`をキーとして内部結合すると、以下のようになります。`on`をキーとして設定します。なお、以下の結合方法はデフォルトで内部結合になります。

In [30]:
# データのマージ（内部結合、inner　join が省略されてる、またキーは自動的に認識されるが、onで明示的に指定可能）
# また複数キーも可能、リストで指定
print("・結合テーブル")
pd.merge(attri_data_frame1,attri_data_frame2,on='ID')

・結合テーブル


,ID,birth_year,city,name,English,index_num,math,sex
0,100,1990,Tokyo,Hiroshi,90,0,50,M
1,101,1989,Osaka,Akiko,30,1,43,F
2,102,1992,Kyoto,Yuki,20,2,33,F


`ID`の値が両方の`dataframe`に存在するものみが表示されました。

#### 左外部結合
次の例は、左側のテーブルに合わせて、`dataframe`のデータを結合するものです。左側に対応するデータが右にない場合は、`NaN`になります。これが左外部結合です。

In [31]:
# データのマージ（left）
pd.merge(attri_data_frame1,attri_data_frame2,how='left')

,ID,birth_year,city,name,English,index_num,math,sex
0,100,1990,Tokyo,Hiroshi,90.0,0.0,50.0,M
1,101,1989,Osaka,Akiko,30.0,1.0,43.0,F
2,102,1992,Kyoto,Yuki,20.0,2.0,33.0,F
3,103,1997,Hokkaido,Satoru,NaN,NaN,NaN,NaN
4,104,1982,Tokyo,Steeve,NaN,NaN,NaN,NaN
5,106,1991,Tokyo,Mituru,NaN,NaN,NaN,NaN
6,108,1988,Osaka,Aoi,NaN,NaN,NaN,NaN
7,110,1990,Kyoto,Tarou,NaN,NaN,NaN,NaN
8,111,1995,Hokkaido,Suguru,NaN,NaN,NaN,NaN
9,113,1981,Tokyo,Mitsuo,NaN,NaN,NaN,NaN


#### 全結合
次の例は、どちらのデータにも存在するデータで結合しています。これが全結合です。値がない場合は、`NaN`になります。

In [32]:
# データのマージ（outer）
pd.merge(attri_data_frame1,attri_data_frame2,how='outer')

,ID,birth_year,city,name,English,index_num,math,sex
0,100,1990.0,Tokyo,Hiroshi,90.0,0.0,50.0,M
1,101,1989.0,Osaka,Akiko,30.0,1.0,43.0,F
2,102,1992.0,Kyoto,Yuki,20.0,2.0,33.0,F
3,103,1997.0,Hokkaido,Satoru,NaN,NaN,NaN,NaN
4,104,1982.0,Tokyo,Steeve,NaN,NaN,NaN,NaN
5,106,1991.0,Tokyo,Mituru,NaN,NaN,NaN,NaN
6,108,1988.0,Osaka,Aoi,NaN,NaN,NaN,NaN
7,110,1990.0,Kyoto,Tarou,NaN,NaN,NaN,NaN
8,111,1995.0,Hokkaido,Suguru,NaN,NaN,NaN,NaN
9,113,1981.0,Tokyo,Mitsuo,NaN,NaN,NaN,NaN


なお、キーをインデックスで指定して結合することもできます。

In [ ]:
# index によるマージ
pd.merge(attri_data_frame1,attri_data_frame2,left_index=True,right_on='index_num')

#### 縦結合

これまでは、何らかのキーに紐付いてデータをマージしていましたが、`concat`を使うと、データを縦方向の積み上げられます。これを縦結合と言います。

In [34]:
# データの準備
attri_data3 = {'ID':['117','118','119','120','125']
        ,'city':['Chiba','Kanagawa','Tokyo','Fukuoka','Okinawa']
        ,'birth_year':[1990,1989,1992,1997,1982]
        ,'name':['Suguru','Kouichi','Satochi','Yukie','Akari']}
attri_data_frame3 = DataFrame(attri_data3)


In [35]:
# concat 縦結合
concat_data = pd.concat([attri_data_frame1,attri_data_frame3])
# 注意：カラムがないとNaNになる
concat_data

,ID,birth_year,city,name
0,100,1990,Tokyo,Hiroshi
1,101,1989,Osaka,Akiko
2,102,1992,Kyoto,Yuki
3,103,1997,Hokkaido,Satoru
4,104,1982,Tokyo,Steeve
5,106,1991,Tokyo,Mituru
6,108,1988,Osaka,Aoi
7,110,1990,Kyoto,Tarou
8,111,1995,Hokkaido,Suguru
9,113,1981,Tokyo,Mitsuo


#### <練習問題 1>

下記の2つのデータテーブルに対して、内部結合してみましょう。

In [36]:
# データ4の準備
attri_data4 = {'ID':['0','1','2','3','4','6','8','11','12','13']
        ,'city':['Tokyo','Osaka','Kyoto','Hokkaido','Tokyo','Tokyo','Osaka','Kyoto','Hokkaido','Tokyo']
        ,'birth_year':[1990,1989,1992,1997,1982,1991,1988,1990,1995,1981]
        ,'name':['Hiroshi','Akiko','Yuki','Satoru','Steeve','Mituru','Aoi','Tarou','Suguru','Mitsuo']}
attri_data_frame4 = DataFrame(attri_data4)
attri_data_frame4

,ID,birth_year,city,name
0,0,1990,Tokyo,Hiroshi
1,1,1989,Osaka,Akiko
2,2,1992,Kyoto,Yuki
3,3,1997,Hokkaido,Satoru
4,4,1982,Tokyo,Steeve
5,6,1991,Tokyo,Mituru
6,8,1988,Osaka,Aoi
7,11,1990,Kyoto,Tarou
8,12,1995,Hokkaido,Suguru
9,13,1981,Tokyo,Mitsuo


In [37]:
# データ5の準備
attri_data5 = {'ID':['0','1','3','6','8']
        ,'math':[20,30,50,70,90]
        ,'English':[30,50,50,70,20]
        ,'sex':['M','F','F','M','M']
        ,'index_num':[0,1,2,3,4]}
attri_data_frame5 = DataFrame(attri_data5)
attri_data_frame5

,English,ID,index_num,math,sex
0,30,0,0,20,M
1,50,1,1,30,F
2,50,3,2,50,F
3,70,6,3,70,M
4,20,8,4,90,M


#### <練習問題 2>

`attri_data_frame4`をベースに`attri_data_frame5`のテーブルを外部結合してみましょう。

#### <練習問題 3>
`attri_data_frame4`に対して、以下のデータを縦結合してみましょう。

In [38]:
# データの準備
attri_data6 = {'ID':['70','80','90','120','150']
        ,'city':['Chiba','Kanagawa','Tokyo','Fukuoka','Okinawa']
        ,'birth_year':[1980,1999,1995,1994,1994]
        ,'name':['Suguru','Kouichi','Satochi','Yukie','Akari']}
attri_data_frame6 = DataFrame(attri_data6)

### 6.2.3 データの操作と変換
キーワード：データのピボット操作、重複データ、マッピング、ビン分割

次に、データの操作と変換（ピボット操作、データの重複があった場合の処理、マッピング、ビン分割など）について扱っていきましょう。

### ピボット操作
まずは、データのピボット操作について学びます。もう一度、これまで使ってきた階層テーブル`hier_data_frame`を例にとして考えます。

In [5]:
hier_data_frame = DataFrame(np.arange(9).reshape((3,3))
                           ,index = [['a','a','b'],[1,2,2]]
                           ,columns = [['Osaka','Tokyo','Osaka']
                                      ,['Blue','Red','Red']]
                           )
hier_data_frame

Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

次のように`stack`を実行すると、行と列が入れ替わった`DataFrame`を再構成できます。

In [6]:
#　ピボット　列が行に 
hier_data_frame.stack()

Osaka  Tokyo
a 1 Blue      0    NaN
    Red       2    1.0
  2 Blue      3    NaN
    Red       5    4.0
b 2 Blue      6    NaN
    Red       8    7.0

`unstack`を使うと、逆の操作が可能です。

In [7]:
# 再配置
hier_data_frame.stack().unstack()

Osaka     Tokyo     
     Blue Red  Blue  Red
a 1     0   2   NaN  1.0
  2     3   5   NaN  4.0
b 2     6   8   NaN  7.0

上記のデータ操作では、縦にあったものを横に持ってきたり、横にあったものを縦に持ってきたりしており、これらのテクニックは、データのモデリング前の処理として使うことも多く便利ですので、ぜひ理解して使えるようになってください。

### 重複データの除去
次は、重複があるデータの処理です。データ分析をしていると、データに重複があることもありますし、自分で実際に集計等していて重複が混じることもあり、そのチェックをするという意味で重要です。

まず、重複するデータを準備しましょう。

In [8]:
#　重複データ
dupli_data = DataFrame({'col1':[1,1,2,3,4,4,6,6]
                       ,'col2':['a','b','b','b','c','c','b','b']})
print("・元のデータ")
dupli_data

・元のデータ


,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
5,4,c
6,6,b
7,6,b


重複の判定には`duplicated()`を使います。それぞれの行が確認され、重複がある場合ときは、`True`となります。

In [42]:
#　重複判定
print("・重複ありの行")
dupli_data.duplicated()

・重複ありの行


0    False
1    False
2    False
3    False
4    False
5     True
6    False
7     True
dtype: bool

`drop_duplicates()`を使うと、重複したデータを削除した結果のデータが返されます。

In [43]:
#　重複削除
print("・重複削除後のデータ")
dupli_data.drop_duplicates()

・重複削除後のデータ


,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
6,6,b


### マッピング処理
次は、マッピング処理です。これは、Excelの関数のvlookupのような処理です。共通のキーとなるデータに対して、一方の（参照）テーブルからそのキーに対応するデータを引っ張ってくる機能です。以下は、都道府県名と地域名を対応付けた参照データです。次のように変換するデータとして使います。

Tokyo（東京）→Kanto（関東）

Hokkaido（北海道）→Hokkaido（北海道）

Osaka（大阪）→Kansai（関西）

Kyoto（京都）→Kansai（関西）


In [44]:
# 参照データ
city_map ={'Tokyo':'Kanto'
          ,'Hokkaido':'Hokkaido'
          ,'Osaka':'Kansai'
          ,'Kyoto':'Kansai'}
city_map

{'Hokkaido': 'Hokkaido',
 'Kyoto': 'Kansai',
 'Osaka': 'Kansai',
 'Tokyo': 'Kanto'}

次の例は、`attri_data_frame1`の`city`カラムをベースとして、上の参照データに対応する地域名データを持ってきて、新しく一番右に`region`というカラムとして追加するものです。

In [45]:
#　参照データを結合
# もし対応するデータがなかったら、NaNになる。
attri_data_frame1['region'] = attri_data_frame1['city'].map(city_map)
attri_data_frame1

,ID,birth_year,city,name,region
0,100,1990,Tokyo,Hiroshi,Kanto
1,101,1989,Osaka,Akiko,Kansai
2,102,1992,Kyoto,Yuki,Kansai
3,103,1997,Hokkaido,Satoru,Hokkaido
4,104,1982,Tokyo,Steeve,Kanto
5,106,1991,Tokyo,Mituru,Kanto
6,108,1988,Osaka,Aoi,Kansai
7,110,1990,Kyoto,Tarou,Kansai
8,111,1995,Hokkaido,Suguru,Hokkaido
9,113,1981,Tokyo,Mitsuo,Kanto


このように新しい変数`region`をつけることで、この単位で集計が可能になります。

#### 無名関数とmapを組み合わせる

次は、Pythonの基礎で学んだ無名関数と`map`を使って、カラムの中の一部のデータを取り出す処理をする例です。具体的には、`birth_year`の上3桁を取得します。関数適応やループなどを使って要素を1つ1つ取り出して処理するより便利なので、まとめて処理したい場合は、このようなやり方を検討することをおすすめします。

In [46]:
#　birth_year の上3つの数字・文字を取り出す
attri_data_frame1['up_two_num'] = attri_data_frame1['birth_year'].map(lambda x:str(x)[0:3])
attri_data_frame1

,ID,birth_year,city,name,region,up_two_num
0,100,1990,Tokyo,Hiroshi,Kanto,199
1,101,1989,Osaka,Akiko,Kansai,198
2,102,1992,Kyoto,Yuki,Kansai,199
3,103,1997,Hokkaido,Satoru,Hokkaido,199
4,104,1982,Tokyo,Steeve,Kanto,198
5,106,1991,Tokyo,Mituru,Kanto,199
6,108,1988,Osaka,Aoi,Kansai,198
7,110,1990,Kyoto,Tarou,Kansai,199
8,111,1995,Hokkaido,Suguru,Hokkaido,199
9,113,1981,Tokyo,Mitsuo,Kanto,198


### ビン分割

最後にビン分割について説明します。これは、ある離散的な範囲にデータを分割して集計したい場合に、便利な機能です。具体的には、上のデータの`birth_year`に対して、5年区切りで集計をしたい場合や、ある特定の分割で計算をしたい場合に使います。
たとえば以下のように、1980、1985、1990、1995、2000のように5年単位でビン分割するためのリストを用意し、pandasの`cut`関数を使うと、そのように分割できます。

In [47]:
#　分割の粒度
birth_year_bins = [1980,1985,1990,1995,2000]

# ビン分割の実施
birth_year_cut_data = pd.cut(attri_data_frame1.birth_year,birth_year_bins)
birth_year_cut_data

0    (1985, 1990]
1    (1985, 1990]
2    (1990, 1995]
3    (1995, 2000]
4    (1980, 1985]
5    (1990, 1995]
6    (1985, 1990]
7    (1985, 1990]
8    (1990, 1995]
9    (1980, 1985]
Name: birth_year, dtype: category
Categories (4, interval[int64]): [(1980, 1985] < (1985, 1990] < (1990, 1995] < (1995, 2000]]

上記の結果を使いそれぞれの数を集計したい場合は、`value_counts`を使います。

In [48]:
# 集計結果
pd.value_counts(birth_year_cut_data)

(1985, 1990]    4
(1990, 1995]    3
(1980, 1985]    2
(1995, 2000]    1
Name: birth_year, dtype: int64

それぞれのビンに名前をつけることもできます。

In [49]:
# 名前付き
group_names = ["early1980s", "late1980s", "early1990s", "late1990s"]
birth_year_cut_data = pd.cut(attri_data_frame1.birth_year,birth_year_bins,labels = group_names)
pd.value_counts(birth_year_cut_data)

late1980s     4
early1990s    3
early1980s    2
late1990s     1
Name: birth_year, dtype: int64

上記では、ビン分割のリストを用意しましたが、あらかじめ分割数を指定したい場合は、以下のように設定できます。

In [50]:
# 数字で分割数指定可能。
# ここでは2つに分割
pd.cut(attri_data_frame1.birth_year,2)

0      (1989.0, 1997.0]
1    (1980.984, 1989.0]
2      (1989.0, 1997.0]
3      (1989.0, 1997.0]
4    (1980.984, 1989.0]
5      (1989.0, 1997.0]
6    (1980.984, 1989.0]
7      (1989.0, 1997.0]
8      (1989.0, 1997.0]
9    (1980.984, 1989.0]
Name: birth_year, dtype: category
Categories (2, interval[float64]): [(1980.984, 1989.0] < (1989.0, 1997.0]]

`qcut`を使うと、分位点での分割もできます。`qcut`を使えば、ほぼ同じサイズのビンを作成することができます。

In [51]:
pd.value_counts(pd.qcut(attri_data_frame1.birth_year,2))

(1980.999, 1990.0]    6
(1990.0, 1997.0]      4
Name: birth_year, dtype: int64

このビン分割は、具体的には、顧客の購買金額合計を分けて分析をしたい場合など、マーケティング分析にも使えます。詳しいことは、総合問題演習で扱っていくことにしましょう。

#### <練習問題 1>

以前の章で使用した「student-mat.csv」のデータを使い、`age`を2倍にしたカラムを追加してみましょう。

In [ ]:
# chapter3で用意したデータがあるpathに移動して、以下を実行してください。例） cd pathの名前
student_data_math = pd.read_csv("student-mat.csv",sep=";")

#### <練習問題 2>

上記と同じデータで、「`absences`」のカラムについて、以下の3つのビンに分けてそれぞれの人数を数えてみましょう。なお、`cut`のオプション設定で、デフォルトは右側が閉区間になっていますが、今回は0を含むため、`right=False`を追加してください。

In [53]:
#　分割の粒度
absences_bins = [0,1,5,100]

#### <練習問題 3>

上記と同じデータで、「`absences`」のカラムについて、`qcut`を用いて3つのビンに分けてみましょう。

### 6.2.4 データの集約とグループ演算
キーワード：group by

ここでは、あるカラムを軸にして、集計する処理を学びます。2章で少し扱いましたが、`group by`を使うことで、ある変数を軸として、その単位で集計処理をします。以前使った`attri_data_frame1`データを対象に、集約やグループ演算をしたいと思います。

In [79]:
attri_data_frame1

,ID,birth_year,city,name,region,up_two_num
0,100,1990,Tokyo,Hiroshi,Kanto,199
1,101,1989,Osaka,Akiko,Kansai,198
2,102,1992,Kyoto,Yuki,Kansai,199
3,103,1997,Hokkaido,Satoru,Hokkaido,199
4,104,1982,Tokyo,Steeve,Kanto,198
5,106,1991,Tokyo,Mituru,Kanto,199
6,108,1988,Osaka,Aoi,Kansai,198
7,110,1990,Kyoto,Tarou,Kansai,199
8,111,1995,Hokkaido,Suguru,Hokkaido,199
9,113,1981,Tokyo,Mitsuo,Kanto,198


以下のようにすると、それぞれの`city`の値がいくつかあるのかを計算できます。

In [54]:
# サイズ情報
attri_data_frame1.groupby("city").size()

city
Hokkaido    2
Kyoto       2
Osaka       2
Tokyo       4
dtype: int64

次は、`city`を軸としてに、`birth_year`の平均値を算出する例です。

In [55]:
# Cityを軸に、birth_yearの平均値を求める
attri_data_frame1.groupby("city")["birth_year"].mean()

city
Hokkaido    1996.0
Kyoto       1991.0
Osaka       1988.5
Tokyo       1986.0
Name: birth_year, dtype: float64

軸は複数設定することもできます。たとえば、`region`、`City`を2軸として、`birth_year`の平均値を求めると、次のようになります。

In [56]:
attri_data_frame1.groupby(["region","city"])["birth_year"].mean()

region    city    
Hokkaido  Hokkaido    1996.0
Kansai    Kyoto       1991.0
          Osaka       1988.5
Kanto     Tokyo       1986.0
Name: birth_year, dtype: float64

なお、as_index=Falseにすると、インデックスが設定されなくなります。そのままテーブルとして扱いたいときに便利です。

In [57]:
attri_data_frame1.groupby(["region","city"],as_index=False)["birth_year"].mean()

,region,city,birth_year
0,Hokkaido,Hokkaido,1996.0
1,Kansai,Kyoto,1991.0
2,Kansai,Osaka,1988.5
3,Kanto,Tokyo,1986.0


他にも`groupby`には、`iteratio`nの機能があり、次のように結果の要素をPythonの`for`などでループ処理できて便利です。

In [89]:
for group, subdf in attri_data_frame1.groupby("region"):
    print(group)
    print(subdf)

Hokkaido
    ID  birth_year      city    name    region up_two_num
3  103        1997  Hokkaido  Satoru  Hokkaido        199
8  111        1995  Hokkaido  Suguru  Hokkaido        199
Kansai
    ID  birth_year   city   name  region up_two_num
1  101        1989  Osaka  Akiko  Kansai        198
2  102        1992  Kyoto   Yuki  Kansai        199
6  108        1988  Osaka    Aoi  Kansai        198
7  110        1990  Kyoto  Tarou  Kansai        199
Kanto
    ID  birth_year   city     name region up_two_num
0  100        1990  Tokyo  Hiroshi  Kanto        199
4  104        1982  Tokyo   Steeve  Kanto        198
5  106        1991  Tokyo   Mituru  Kanto        199
9  113        1981  Tokyo   Mitsuo  Kanto        198


要素に対して、複数の異なる値をまとめて算出したいときには、`agg`を使うと便利です。`agg`の引数には、実行したい関数名のリストを渡します。以下は、カウント、平均、最大、最小を計算する例です。

In [ ]:
# 列に複数の関数を適応
functions = ['count','mean','max','min']
grouped_student_math_data1 = student_data_math.groupby(['sex','address'])
grouped_student_math_data1['age','G1'].agg(functions)

#### <練習問題 1>

先ほど使用した「student-mat.csv」を使って「student-mat.csv」を使って、pandasの集計処理してみましょう。まずは、`school`を軸にして、G1の平均点をそれぞれ求めてみましょう。

#### <練習問題 2>

次は、`school`と性別を軸にして、G1、G2、G3の平均点をそれぞれ求めてみましょう。

#### <練習問題 3>

次は、`school`と性別を軸にして、G1、G2、G3の最大値、最小値をまとめて算出してみましょう。

***

## 6.3 欠損データと異常値の取り扱いの基礎
ゴール：欠損データと異常値に対する基本的な対処方法を知る

データを扱っていると必ずといっていいほど、欠損しているデータや異常値データの存在があります。この章では基礎の基礎レベルで欠損データや異常データについての判定や扱い方について学ぶことにします。もっと深く学びたい方は、ぜひ参考文献「A-12」を読んでください。

### 6.3.1 欠損データの扱い方
キーワード：リストワイズ削除、ペアワイズ削除、平均値代入法

まずは、欠損データの取り扱いについてです。データの欠損は、入力忘れ、無回答、システム上の問題など様々な要因があります。「ない」データについては、無視をするのがいいのか、除外をするのがいいのか、最もらしい値を入れるのがいいのか、それが問題です。アプローチによっては、大きなバイアスのある結果を与え、誤った意思決定につながり、大きな損失につながる可能性もあります。慎重に扱っていきましょう。

この節では、次のようなデータをサンプルとして扱います。値を`NaN`（`NA`）にした部分が欠損データであるとして、以下、説明を続けます。

In [59]:
# データの準備
import numpy as np
from numpy import nan as NA
import pandas as pd


sample_data_frame = pd.DataFrame(np.random.rand(10,4))

# NAにする
sample_data_frame.iloc[1,0] = NA
sample_data_frame.iloc[2:3,2] = NA
sample_data_frame.iloc[5:,3] = NA

In [60]:
sample_data_frame

,0,1,2,3
0,0.893145,0.461451,0.447737,0.130427
1,NaN,0.501456,0.462464,0.834043
2,0.092436,0.573044,NaN,0.277006
3,0.003648,0.241355,0.469863,0.622194
4,0.882922,0.955569,0.143864,0.521940
5,0.020924,0.306699,0.934956,NaN
6,0.044159,0.428740,0.934661,NaN
7,0.615847,0.108205,0.949121,NaN
8,0.761155,0.010232,0.420065,NaN
9,0.336717,0.277855,0.271509,NaN


以下では、この擬似的な欠損データに対して、削除や、0や直前の数字、平均値等で穴埋めをしています。本書では、これらの単純な方法のみ紹介しますが、他の方法には最尤推定法で推定したり、回帰代入やScipyで実施したスプライン補間などがあります。注意が必要なのは、これらの方法がバイアスを生む可能性があることです。深く学びたい方はぜひ上で紹介した参考文献「A-12」などを読んで、欠損データを埋める方法への理解を深めてください。

#### リストワイズ削除
NaNがある行をすべて取り除くには、`dropna`を使います。これを**リストワイズ削除**といいます。
以下は、先ほどのデータにおいて、`dropna`を適用し、すべてのカラムにデータがある行だけを抽出したものです。`NaN`がある行は除外されます。

In [61]:
sample_data_frame.dropna()

,0,1,2,3
0,0.893145,0.461451,0.447737,0.130427
3,0.003648,0.241355,0.469863,0.622194
4,0.882922,0.955569,0.143864,0.521940


#### ペアワイズ削除
この結果からわかるように、リストワイズ削除では元々10行あったデータが極端に少なくなって、データが全く使えないという状況が考えられます。このとき、欠損している列のデータを無視して、利用可能なデータのみ（例：列の0番目と1番目のみ存在）を使う方法があります。これを**ペアワイズ削除**といいます。ペアワイズ削除では、使いたい列を取り出してから`dropna`します。

In [62]:
sample_data_frame[[0,1]].dropna()

,0,1
0,0.893145,0.461451
2,0.092436,0.573044
3,0.003648,0.241355
4,0.882922,0.955569
5,0.020924,0.306699
6,0.044159,0.428740
7,0.615847,0.108205
8,0.761155,0.010232
9,0.336717,0.277855


#### fillnaで埋める
他の処理として、`fillna`（値）で、`NaN`になっている箇所をある値で埋める方法もあります。たとえば`NaN`を0として扱うケースです。次のように`fillna(0)`とすると、`NaN`が0に置き変わります。

In [63]:
sample_data_frame.fillna(0)

,0,1,2,3
0,0.893145,0.461451,0.447737,0.130427
1,0.000000,0.501456,0.462464,0.834043
2,0.092436,0.573044,0.000000,0.277006
3,0.003648,0.241355,0.469863,0.622194
4,0.882922,0.955569,0.143864,0.521940
5,0.020924,0.306699,0.934956,0.000000
6,0.044159,0.428740,0.934661,0.000000
7,0.615847,0.108205,0.949121,0.000000
8,0.761155,0.010232,0.420065,0.000000
9,0.336717,0.277855,0.271509,0.000000


#### 前の値で埋める
`method`に`ffill`を指定することで、前の値で埋めることもできます。具体的には、2行1列目は先ほど

`sample_data_frame.iloc[1,0] = NA`

で`NA`にしましたが、手前の1行1列目の値は、0.893145でしたので、この値で埋めることができます。この処理は金融の時系列データの処理などで使うことができ、便利です。

In [64]:
sample_data_frame.fillna(method="ffill")

,0,1,2,3
0,0.893145,0.461451,0.447737,0.130427
1,0.893145,0.501456,0.462464,0.834043
2,0.092436,0.573044,0.462464,0.277006
3,0.003648,0.241355,0.469863,0.622194
4,0.882922,0.955569,0.143864,0.521940
5,0.020924,0.306699,0.934956,0.521940
6,0.044159,0.428740,0.934661,0.521940
7,0.615847,0.108205,0.949121,0.521940
8,0.761155,0.010232,0.420065,0.521940
9,0.336717,0.277855,0.271509,0.521940


#### 平均値で埋める
他に、平均値でも穴埋めする方法もあります。これを**平均値代入法**といい、`mean`を使います。

In [65]:
# 各カラムの平均値(確認用)
sample_data_frame.mean()

0    0.405661
1    0.386461
2    0.559360
3    0.477122
dtype: float64

In [66]:
sample_data_frame.fillna(sample_data_frame.mean())

,0,1,2,3
0,0.893145,0.461451,0.447737,0.130427
1,0.405661,0.501456,0.462464,0.834043
2,0.092436,0.573044,0.559360,0.277006
3,0.003648,0.241355,0.469863,0.622194
4,0.882922,0.955569,0.143864,0.521940
5,0.020924,0.306699,0.934956,0.477122
6,0.044159,0.428740,0.934661,0.477122
7,0.615847,0.108205,0.949121,0.477122
8,0.761155,0.010232,0.420065,0.477122
9,0.336717,0.277855,0.271509,0.477122


他にも色々とオプションがあるので、`?sample_data_frame.fillna`等で調べてみてください。

欠損データについて、ここではサンプルデータにおいて、一定の値を機械的に置換しました。ただし、これらの方法はいつも使えるというわけではありません。データの状況、背景等を考え、適切に対処することが重要です。

#### <練習問題 1>

以下のデータに対して、1列でも`NaN`がある場合は削除し、その結果を表示してください。

In [67]:
# データの準備
import numpy as np
from numpy import nan as NA
import pandas as pd


sample_data_frame2 = pd.DataFrame(np.random.rand(15,6))

# NAにする
sample_data_frame2.iloc[2,0] = NA
sample_data_frame2.iloc[5:8,2] = NA
sample_data_frame2.iloc[7:9,3] = NA
sample_data_frame2.iloc[10,5] = NA


sample_data_frame2

,0,1,2,3,4,5
0,0.247690,0.534316,0.385569,0.208775,0.301835,0.415085
1,0.954748,0.234037,0.969279,0.746641,0.047448,0.026800
2,NaN,0.352614,0.300492,0.263359,0.162665,0.446953
3,0.713572,0.367785,0.836540,0.672065,0.665018,0.907532
4,0.365903,0.601725,0.207926,0.661296,0.844857,0.691277
5,0.783184,0.414782,NaN,0.483368,0.916126,0.932082
6,0.308639,0.535105,NaN,0.062036,0.108739,0.660797
7,0.323623,0.892262,NaN,NaN,0.225616,0.358252
8,0.554777,0.924564,0.226030,NaN,0.024755,0.106058
9,0.701810,0.806429,0.095198,0.469107,0.848404,0.311082


#### <練習問題 2>

上記で準備したデータに対して、`NaN`を0で埋めてください。

#### <練習問題 3>

上記で準備したデータに対して、`NaN`をそれぞれの列の平均値で埋めてください。

### 6.3.2 異常データの扱い方
キーワード：異常値、箱ひげ図、パーセンタイル、VaR（Value At Risk）

異常値（外れ値）データの扱いは、そのままにして何もしないのか、異常値を除去するか、もっともらしい値に入れかえて使うかが問題になります。
そもそも異常値とは一体何でしょうか。実は、統一的な見解というものはなく、そのデータを扱うアナリストや意思決定者が判断することもあります。ビジネスの現場では、不正アクセスのパターン（セキュリティ分野）や機械の故障、金融リスク管理（VaR）など、様々な分野で使われており、それぞれ色々な方法でアプローチされています。

異常値検出のアプローチには、単純には箱ひげ図などを書いて、あるパーセンタイル以上のデータを異常値としてみなす方法、正規分布を利用する方法、データの空間的な近さに基づく方法などがあります。他には以降の章で学ぶ機械学習（教師なし学習も含む）を用いた方法があります。

ここでは特に練習問題はありませんが、興味のある方はぜひ巻末の参考文献「A-13」や参考URL「B-22」などで学んでください。

また、異常値の分野に関連して、極端な値を研究する極値統計学という分野もあります。データの中で大きな値をとる極値データの挙動について、さまざまな研究がなされており、稀ではありますがそれが起きれば非常に大きな影響を及ぼす現象（自然現象、災害など）を研究します。気象学だけではなく、ファイナンスや情報通信の分野でも応用されているので、興味のある方は参考文献「A-14」などを参照してみてください。

以上で、欠損値と異常値の扱いについてはこれで終わりになります。データ分析において、データの前処理が8割だとよく言われます。世の中には実に様々な形式のデータが存在し、それらを整えるだけでも大変な作業です。テクニックも重要ですが、それらに対してどのように対処していくのか戦略を立てることも重要です。参考文献「A-15」にも、ぜひ目を通してみてください。

***

## 6.4 時系列データの取り扱いの基礎
ゴール：Pandasを使って、時系列データの基本的な扱い方を身に付ける

この章の最後にPandasを使った時系列データの取り扱いについて学びます。ここでは、サンプルとして為替の時系列データを扱います。
あらかじめ`pandas-datareader`というライブラリをダウンロードしてインストールしてください。

次のコマンドでpandas-datareaderがインストールできます。

In [ ]:
!pip install pandas-datareader

インストールしたら、次のようにインポートしてください。

In [4]:
import pandas_datareader.data as pdr

### 6.4.1 時系列データの処理と変換
キーワード：リサンプリング、シフト

ここでは、サンプルデータに含まれる2001/1/2から2016/12/30までのDEXJPUSのデータを使います。日ごとのレートデータで、欠損している日（休日など）もあります。

In [ ]:
start_date = "2001/1/2"
end_date = "2016/12/30"

fx_jpusdata = pdr.DataReader("DEXJPUS","fred",start_date,end_date)

In [69]:
fx_jpusdata.head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-08,115.97


サンプルには、15年分のデータがありますが、これをどう分析するかはそのビジネスニーズ次第です。たとえば、最後の2016年の4月のデータだけ欲しいこともありますし、月末のレートだけを見たいこともあります。さらに、上記では、2001/1/6はデータとしてありませんが、それを前日の値で埋めたいこともありますし、前の日と比べてどれだけレートが上がったのか調べたい場合もあるでしょう。これらのことはすべてPandasで簡単に計算することができます。

#### 特定の年月のデータを参照する

まずは、特定の年月のデータを参照する方法です。2016年の4月のデータだけ見たい場合は、以下のように年月を指定して、以下のようにします。

In [70]:
fx_jpusdata['2016-04']

,DEXJPUS
DATE,
2016-04-01,112.06
2016-04-04,111.18
2016-04-05,110.26
2016-04-06,109.63
2016-04-07,107.98
2016-04-08,108.36
2016-04-11,107.96
2016-04-12,108.54
2016-04-13,109.21


他、特定の年や日にちにだけ抽出することができます。
次に、月末レートだけ取り出すことをやってみます。パラメータとして`resample`のあとに`M`を指定することで、月ごとのデータを取り出し、そのあとの`last`は最後のデータを取り出すということをしています。具体的には、以下のデータをみるとわかる通り、1月、2月、3月…の月末のレートを取り出せます。

In [71]:
fx_jpusdata.resample('M').last().head()

,DEXJPUS
DATE,
2001-01-31,116.39
2001-02-28,117.28
2001-03-31,125.54
2001-04-30,123.57
2001-05-31,118.88


パラメータには、日付を取り出したい場合は「`D`」年を取り出したい場合は「`A`」を使います。このように、ある時系列の頻度を別の頻度のデータに変換する処理を**リサンプリング**といいます。また、最後のデータではなく、その平均を計算したい場合は`mean`を使うことで計算できます。他にもいろいろとパラメータを設定できるので、必要な処理があるときに、調べてみてください。

#### 欠損がある場合の操作

次に、時系列データに欠損がある場合の処理をみていきます。欠損処理については、前の節でも扱った通り、さまざまな方法があります。先ほどのレートでは、2001/1/6がまずレコードとして存在していませんでしたが、日ごとにデータを用意したいときは、先ほどのリサンプリングを行います。具体的には、以下のようにします。

In [72]:
fx_jpusdata.resample('D').last().head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-06,NaN


上記より、2001/1/6は空のままなので、前の日の値で埋める処理をします。ここでは、次に示すようにffillを使います。

In [73]:
fx_jpusdata.resample('D').ffill().head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-06,116.19


#### データをズラして比率を計算する

次に、前日とのレート比較をしたい場合を考えます。上のデータ例でいうと、2001-01-02のレートは114.73で、2001-01-03のレートは114.26になり、その比率を計算することもできますが、それをすべての日付について適応させる処理をします。shiftを使うことで、インデックスは固定されて、そのデータだけをずらすことができます。ここでは`shift`関数を使います。以下はデータを1つあとにずらしており、2001-01-02のレートは114.73でしたが、2001-01-03のレートとして扱われるようになります。

In [74]:
fx_jpusdata.shift(1).head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,114.73
2001-01-04,114.26
2001-01-05,115.47
2001-01-08,116.19


このように加工すると、前日のレートと当日のレートの比率を一気に算出することができます。これがPandasを使うメリットです。なお、以下で2001-01-02が`NaN`になっているのは、その前日のデータがもともとないためです。

In [75]:
fx_jpusdata_ratio = fx_jpusdata / fx_jpusdata.shift(1)
fx_jpusdata_ratio.head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,0.995903
2001-01-04,1.010590
2001-01-05,1.006235
2001-01-08,0.998107


#### <練習問題 1>

先ほど使用した`fx_jpusdata`を使って、年ごとの平均値の推移データを作成してください。

### 6.4.2 移動平均
キーワード：移動平均

次に、時系列のデータ処理でよく使われる移動平均の処理方法をみていきます。さきほど扱った`fx_jpusdata`のデータについて、3日間の移動平均線を作成することを考えます。

In [76]:
fx_jpusdata.head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-08,115.97


具体的には、2001-01-04までの平均は2001-01-02が114.73、2001-01-03が114.26、2001-01-04が115.47になるため、その平均を計算すると114.82になります。それを2001-01-05、2001-01-06と続けて計算をしていくのですが、pandasの`rolling`を使うと、簡単に計算できます。以下は、その3日間の移動平均を計算した結果です。`rolling`を実行した後に、`mean`を使って平均を計算しています。

In [77]:
fx_jpusdata.rolling(3).mean().head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,NaN
2001-01-04,114.820000
2001-01-05,115.306667
2001-01-08,115.876667


移動平均ではなく標準偏差の推移を算出したいのなら、`mean`の代わりに`std`を使います。以下は3日間の標準偏差の推移です。

In [78]:
fx_jpusdata.rolling(3).std().head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,NaN
2001-01-04,0.610000
2001-01-05,0.975312
2001-01-08,0.368963


この`rolling`については、パラメータとして他にもいろいろとありますので、必要に応じて調べて実行してみてください。

以上で、Pandasの章は終了です。一部、なかなかイメージを掴みにくい箇所もあったかもしれません。しかし、実際に「こんな感じでデータ加工や変換したいのになあ」と思ったときに、ここを参考にしてプログラミングをしてみてください。データ加工処理のニーズが出てきて、実際に使うことで一層理解が進む箇所かもしれません。また、ここで紹介したテクニックはほんの一部です。この他にも色々なデータ処理・加工方法があるので、参考文献「A-10」などを読んで、手を動かして実行してみてください。

>**[やってみよう]**

>ここで扱った集計軸以外にも、対象データに対していろいろな軸で処理をしてみましょう。

#### <練習問題 1>

先ほど使用した`fx_jpusdata`を使って、20日間の移動平均データを作成してください。ただし`NaN`は削除してください。また、レコードとして存在しないデータであれば、特に補填する必要はありません。

***

## 6.5 総合問題

### ■総合問題1
以前使用した「student-mat.csv」を使って、以下の問いに答えてください。

(1) 上記のデータに対して、年齢×性別でG1の平均点を算出し、縦軸が年齢、横軸が性別となるような表（テーブル）を作成しましょう。

(2) (1)で表示した結果テーブルについて、`NA`になっている行（レコード）をすべて削除した結果を表示しましょう。